## Xây dựng giao diện nhận biết biển báo

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.models import load_model

 

In [2]:
# Tải mô hình đã huấn luyện
cnn_model = load_model('../model/my_model.keras')  
yolo_model = YOLO('runs/detect/train/weights/best.pt')      

 

## Các lớp biển báo 

In [3]:
class_names = { 1:'Giới hạn tốc độ (20km/h)',
            2:'Giới hạn tốc độ (30km/h)',      
            3:'Giới hạn tốc độ (50km/h)',       
            4:'Giới hạn tốc độ (60km/h)',      
            5:'Giới hạn tốc độ (70km/h)',    
            6:'Giới hạn tốc độ (80km/h)',      
            7:'Hết giới hạn tốc độ (80km/h)',     
            8:'Giới hạn tốc độ (100km/h)',    
            9:'Giới hạn tốc độ (120km/h)',     
           10:'Cấm vượt',   
           11:'Cấm xe tải trên 3.5 tấn vượt',     
           12:'Quyền ưu tiên tại ngã tư',     
           13:'Đường ưu tiên',    
           14:'Nhường đường',     
           15:'Dừng lại',       
           16:'Cấm phương tiện',       
           17:'Cấm xe trên 3.5 tấn',       
           18:'Cấm đi ngược chiều',       
           19:'Chú ý chung',     
           20:'Khúc cua nguy hiểm bên trái',      
           21:'Khúc cua nguy hiểm bên phải',   
           22:'Hai khúc cua liên tiếp',      
           23:'Đường gồ ghề',     
           24:'Đường trơn',       
           25:'Đường hẹp bên phải',  
           26:'Công trường',    
           27:'Tín hiệu giao thông',      
           28:'Người đi bộ',     
           29:'Trẻ em băng qua đường',     
           30:'Xe đạp băng qua đường',       
           31:'Cẩn thận băng tuyết',
           32:'Động vật hoang dã băng qua đường',      
           33:'Hết giới hạn tốc độ và cấm vượt',      
           34:'Rẽ phải phía trước',     
           35:'Rẽ trái phía trước',       
           36:'Chỉ được đi thẳng',      
           37:'Đi thẳng hoặc rẽ phải',      
           38:'Đi thẳng hoặc rẽ trái',      
           39:'Đi bên phải',     
           40:'Đi bên trái',      
           41:'Bắt buộc đi theo vòng xuyến',     
           42:'Hết cấm vượt',      
           43:'Hết cấm vượt xe trên 3.5 tấn' }


In [4]:
 


# Thiết kế giao diện và khai báo biến
root = tk.Tk()
root.title("Nhận Biết & Phát Hiện Biển Báo Giao Thông")
root.geometry("800x600")
root.configure(bg="#FFF0F5")

current_result = tk.StringVar()
current_result.set("Không phát hiện biển báo")

# Hàm xử lý ảnh
def process_image(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result_text = "Không phát hiện biển báo"
     
    results = yolo_model(img, conf=0.15, iou=0.5) 
    for result in results:
        boxes = result.boxes.xyxy.numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
           
            roi = img_rgb[y1:y2, x1:x2]
            if roi.size == 0 or (x2 - x1) * (y2 - y1) < 100:  
                print("ROI quá nhỏ hoặc rỗng, bỏ qua vùng này.")
                continue
             
            try:
                roi_resized = cv2.resize(roi, (32,32))
                roi_normalized = roi_resized / 255.0
                roi_input = np.expand_dims(roi_normalized, axis=0)
                
                 
                prediction = cnn_model.predict(roi_input, verbose=0)
                class_idx = np.argmax(prediction)
                confidence = prediction[0][class_idx] * 100
                label = f"{class_names.get(class_idx + 1, 'Không xác định')}: {confidence:.2f}%"
                 
                cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(img_rgb, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                result_text = label
                print(f"Phát hiện: {label}")
            except Exception as e:
                print(f"Lỗi khi xử lý ROI: {e}")
    
    current_result.set(result_text)  # Cập nhật nhãn trên giao diện
    return img_rgb

 

In [5]:
# Hàm xử lý khung hình từ video/webcam
def process_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result_text = "Không phát hiện biển báo"
    
   
    results = yolo_model(frame, conf=0.25, iou=0.25, stream=True)   
    for result in results:
        boxes = result.boxes.xyxy.numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
             
            roi = frame_rgb[y1:y2, x1:x2]
            if roi.size == 0 or (x2 - x1) * (y2 - y1) < 100:   
                print("ROI quá nhỏ hoặc rỗng, bỏ qua vùng này.")
                continue
            
            
            try:
                roi_resized = cv2.resize(roi, (32,32))
                roi_normalized = roi_resized / 255.0
                roi_input = np.expand_dims(roi_normalized, axis=0)
                
               
                prediction = cnn_model.predict(roi_input, verbose=0)
                class_idx = np.argmax(prediction)
                confidence = prediction[0][class_idx] * 100
                label = f"{class_names.get(class_idx + 1, 'Không xác định')}: {confidence:.2f}%"
                 
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                result_text = label
                print(f"Phát hiện: {label}")
            except Exception as e:
                print(f"Lỗi khi xử lý ROI: {e}")
    
    current_result.set(result_text)   
    return frame_rgb

 

In [6]:
# Hàm tải ảnh
def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png")])
    if file_path:
        img = process_image(file_path)
        img = Image.fromarray(img)
        img = img.resize((500, 400), Image.Resampling.LANCZOS)
        photo = ImageTk.PhotoImage(img)
        result_label.config(image=photo)
        result_label.image = photo

# Hàm tải video
def upload_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi")])
    if file_path:
        cap = cv2.VideoCapture(file_path)
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            # Xử lý mỗi 2 khung hình để giảm tải
            if frame_count % 2 == 0:
                # Resize khung hình để tăng tốc độ xử lý
                frame = cv2.resize(frame, (640, 480))  # Giảm kích thước khung hình
                frame_processed = process_frame(frame)
                frame_processed = Image.fromarray(frame_processed)
                frame_processed = frame_processed.resize((500, 400), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(frame_processed)
                result_label.config(image=photo)
                result_label.image = photo
                root.update()
            frame_count += 1
        cap.release()

# Hàm sử dụng webcam
def use_webcam():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize khung hình để tăng tốc độ xử lý
        frame = cv2.resize(frame, (640, 480))  # Giảm kích thước khung hình
        frame_processed = process_frame(frame)
        frame_processed = Image.fromarray(frame_processed)
        frame_processed = frame_processed.resize((500, 400), Image.Resampling.LANCZOS)
        photo = ImageTk.PhotoImage(frame_processed)
        result_label.config(image=photo)
        result_label.image = photo
        root.update()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()

 

In [7]:
# Thiết kế giao diện
title_label = tk.Label(root, text="Nhận Biết & Phát Hiện Biển Báo Giao Thông", font=("Arial", 20, "bold"), bg="#FFB6C1", fg="white")
title_label.pack(pady=10, fill="x")

button_frame = tk.Frame(root, bg="#FFF0F5")
button_frame.pack(pady=10)

upload_img_btn = tk.Button(button_frame, text="Tải Ảnh Lên", command=upload_image, font=("Arial", 12), bg="#FFB6C1", fg="white", relief="flat")
upload_img_btn.grid(row=0, column=0, padx=10)

upload_video_btn = tk.Button(button_frame, text="Tải Video Lên", command=upload_video, font=("Arial", 12), bg="#FFB6C1", fg="white", relief="flat")
upload_video_btn.grid(row=0, column=1, padx=10)

webcam_btn = tk.Button(button_frame, text="Dùng Webcam", command=use_webcam, font=("Arial", 12), bg="#FFB6C1", fg="white", relief="flat")
webcam_btn.grid(row=0, column=2, padx=10)

result_frame = tk.Frame(root, bg="#FFFFFF", bd=2, relief="solid")
result_frame.pack(pady=20, padx=20, fill="both", expand=True)

result_label = tk.Label(result_frame, bg="#FFFFFF")
result_label.pack()

result_text_label = tk.Label(root, textvariable=current_result, font=("Arial", 14), bg="#FFF0F5", fg="#FF69B4")
result_text_label.pack(pady=10)

root.mainloop()


0: 224x224 1 Cm xe trn 3.5 tn, 70.4ms
Speed: 1.1ms preprocess, 70.4ms inference, 2.0ms postprocess per image at shape (1, 3, 224, 224)
Phát hiện: Cấm xe trên 3.5 tấn: 100.00%

0: 192x224 (no detections), 81.0ms
Speed: 1.9ms preprocess, 81.0ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 224)

0: 192x224 (no detections), 51.4ms
Speed: 1.6ms preprocess, 51.4ms inference, 0.8ms postprocess per image at shape (1, 3, 192, 224)

0: 192x224 (no detections), 43.9ms
Speed: 1.3ms preprocess, 43.9ms inference, 0.8ms postprocess per image at shape (1, 3, 192, 224)

0: 192x224 (no detections), 45.6ms
Speed: 1.5ms preprocess, 45.6ms inference, 0.7ms postprocess per image at shape (1, 3, 192, 224)

0: 192x224 1 Ch c i thng, 43.3ms
Phát hiện: Chú ý chung: 79.56%
Speed: 1.8ms preprocess, 43.3ms inference, 1.8ms postprocess per image at shape (1, 3, 192, 224)

0: 192x224 (no detections), 46.6ms
Speed: 1.2ms preprocess, 46.6ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 22

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\OS\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\OS\AppData\Local\Temp\ipykernel_5372\1819671577.py", line 48, in use_webcam
    photo = ImageTk.PhotoImage(frame_processed)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\OS\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\ImageTk.py", line 128, in __init__
    self.__photo = tkinter.PhotoImage(**kw)
                   ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\OS\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 4120, in __init__
    Image.__init__(self, 'photo', name, cnf, master, **kw)
  File "c:\Users\OS\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 4053, in __init__
    master = _get_default_root('create image')
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

Phát hiện: Rẽ trái phía trước: 88.18%
Speed: 1.0ms preprocess, 82.5ms inference, 0.9ms postprocess per image at shape (1, 3, 192, 224)


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\OS\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\OS\AppData\Local\Temp\ipykernel_5372\1819671577.py", line 29, in upload_video
    photo = ImageTk.PhotoImage(frame_processed)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\OS\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\ImageTk.py", line 128, in __init__
    self.__photo = tkinter.PhotoImage(**kw)
                   ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\OS\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 4120, in __init__
    Image.__init__(self, 'photo', name, cnf, master, **kw)
  File "c:\Users\OS\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 4053, in __init__
    master = _get_default_root('create image')
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F